# TP3 - Apartado 1 - Infraestructuras Computacionales para el Procesamiento de Datos Masivos
## Kafka

### 1. Localizar el archivo que contiene la configuración Zookeeper y levantar dicho servidor utilizando la configuración por defecto. ¿Qué puerto es el utilizado por defecto por Zookeeper?

- Archivo: zookeeper.properties
- Comando: `bin/zookeeper-server-start.sh config/zookeeper.properties`
- Puerto por defecto: 2181

### 2. Localizar el archivo que contiene la configuración de un broker y levantar dicho broker. ¿Qué puerto es el utilizado opr defecto por los brokers?

- Archivo: server.properties
- Comando: `bin/kafka-server-start.sh config/server.properties`
- Puerto: 9092

### 3. Crear un topic nuevo con el nombre "initial", con una partición y un factor de replicación igual a 1.

- Comando: `bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic initial`

### 4. Describir el topic "initial" y comentar los resultados que se obtienen.

- Comando: `bin/kafka-topics.sh --bootstrap-server localhost:9092 --describe initial`
- Resultado:

`Topic: initial	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
    Topic: initial	Partition: 0	Leader: 0	Replicas: 0	Isr: `
- Comentario: La primera línea nos da información sobre todas las particiones. En este caso, solo tenemos un topic. 
    - "leader" es el nodo responsable de todas las lecturas y escrituras de dicha partición.
    - "replicas" es la lista de nodos réplicas.
    - "isr" es el número de réplicas que están actualizados con el leader.


### 5. Arrancar un productor sobre el topic initial

- Comando: `bin/kafka-console-producer.sh --broker-list localhost:9092 --topic initial`

### 6. Arrancar un consumidor sobre "initial" que consuma todos los mensajes desde el principio.

- Comando: `bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic intial --from-beginning`

### 7. Generar algunos mensajes en el productor levantado en el ejercicio 5 y observar como se reciben en el consumidor levantado en el ejercicio 6. Comentar la salida obtenida. Detener a continuación el productor y el consumidor.

Para la generación de mensajes simplemente se ha escrito en el productor lo siguientes textos:
```
>hello
>world
>!
```
Lo que ha impreso en la consola del consumidor:
```
hello
world
!
```

Para detener ambos, basta con presionar ^C sobre cada terminal en la que se estaba ejecutando cada uno de ellos.

### 8. Crear un topic nuevo con el nombre "testRep", con dos particiones y un factor de replicación igual a 2. ¿Qué sucede? ¿Cuál es la razón?

- Comando: `bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 2 --partitions 2 --topic testRep`
- Resultado: org.apache.kafka.common.errors.InvalidReplicationFactorException
- Razón: Se ha intentado crear un topic con más réplicas (2) que brokers disponibles (1)

### 9. Detener el broker levantado en el ejercicio 2.

Para detener el broker, basta con ejecutar el siguiente comando: `bin/kafka-server-stop.sh`

### 10. Levantar 3 brokers distintos: generar nuevos ficheros de configuración para cada uno de ellos indicando las modificaciones que han sido necesarias. Los identificadores de los brokers serán 0, 1 y 2.

En primer lugar, creamos tres archivos de configuración distintos, uno para cada broker:
- `cp config/server.properties config/server-0.properties`
- `cp config/server.properties config/server-1.properties`
- `cp config/server.properties config/server-2.properties`

A continuación, en los archivos server-1.properties y server-2.properties, modificamos las diferentes propiedades:
- broker.id -> 1 y 2 respectivamente.
- listeners -> PLAINTEXT://:9094 y PLAINTEXT://:9095 respectivamente.
- log.dirs -> /tmp/kafka-logs-1 y /tmp/kafka-logs-2 respectivamente.

Para arrancar los brokers:

- Archivo: server-0.properties
- Comando: `bin/kafka-server-start.sh config/server-0.properties`
- Puerto: 9092


- Archivo: server-1.properties
- Comando: `bin/kafka-server-start.sh config/server-2.properties`
- Puerto: 9093


- Archivo: server-2.properties
- Comando: `bin/kafka-server-start.sh config/server-2.properties`
- Puerto: 9094

### 11. Describir el topic "initial". Comentar los resultados que se obtienen

Comando: `bin/kafka-topics.sh --describe --bootstrap-server localhost:9092 --topic initial`

Sigue igual que antes, sin replicas y en el servidor (broker) que inicialmente lo habíamos creado (0):

```Topic: initial	PartitionCount: 1	ReplicationFactor: 1	Configs: segment.bytes=1073741824
	Topic: initial	Partition: 0	Leader: 0	Replicas: 0	Isr: 0```


### 12. Crear un topic nuevo con el nombre "testOrder1", con 1 partición y factor de replicación igual a 3.

- Comando: `bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 3 --partitions 1 --topic testOrder1`

### 13. Crear un topic nuevo con el nombre "testOrder2", con 3 particiones y factor de replicación igual a 3.

- Comando: `bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 3 --partitions 3 --topic testOrder2`

### 14. Describir los topics "testOrder1" y "testOrder2" y comentar los resultados que se obtienen.

- Comando: `bin/kafka-topics.sh --describe --bootstrap-server localhost:9092 --topic testOrder1`
- Resultado:

```Topic: testOrder1	PartitionCount: 1	ReplicationFactor: 3	Configs: segment.bytes=1073741824
	Topic: testOrder1	Partition: 0	Leader: 1	Replicas: 1,2,0Isr: 1,2,0```

- Comentarios: Se puede comprobar que hay únicamente una partición, cuyo líder es el broker 1, y cuyas réplicas se encuentran en los brokers 1, 2 y 0.

- Comando: `bin/kafka-topics.sh --describe --bootstrap-server localhost:9092 --topic testOrder2`
- Resultado:

```Topic: testOrder2	PartitionCount: 3	ReplicationFactor: 3	Configs: segment.bytes=1073741824
	Topic: testOrder2	Partition: 0	Leader: 0	Replicas: 0,2,1Isr: 0,2,1
	Topic: testOrder2	Partition: 1	Leader: 2	Replicas: 2,1,0Isr: 2,1,0
	Topic: testOrder2	Partition: 2	Leader: 1	Replicas: 1,0,2Isr: 1,0,2```

- Comentarios: Se puede comprobar que hay 3 particiones (0, 1 y 2) , cuyos líderes son los broker 0, 1 y 2 respectivamente, y cuyas réplicas se encuentran en los brokers 0,2,1 para la partición 0, 2,1,0 para la partición 1 y 1,0,2 para la partición 2.

### 15. Arrancar un productor sobre el topic "testOrder1" y un consumidor desde el inicio sobre el mismo topic. Introducir algunos mensajes y observar como se consumen. Detener el productor y el consumidor.

Comandos:
- `bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testOrder1` 
- `bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --from-beginning --topic testOrder1`


Para la generación de mensajes simplemente se ha escrito en el productor lo siguientes textos:
```
>hello
>world
>!
```
Lo que ha impreso en la consola del consumidor:
```
hello
world
!
```

Para detener ambos, basta con presionar ^C sobre cada terminal en la que se estaba ejecutando cada uno de ellos.

### 16. Arrancar un productor sobre el topic "testOrder2" y un consumidor desde el inicio sobre el mismo topic. Introducir algunos mensajes y observar como se consumen. Detener el productor y el consumidor.

Comandos:
- `bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testOrder2` 
- `bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --from-beginning --topic testOrder2`


Para la generación de mensajes simplemente se ha escrito en el productor lo siguientes textos:
```
>hello
>world
>!
```
Lo que ha impreso en la consola del consumidor:
```
hello
world
!
```

Para detener ambos, basta con presionar ^C sobre cada terminal en la que se estaba ejecutando cada uno de ellos.

### 17. Arrancar un consumidor sobre el topic "testOrder1", sin que consuma desde el principio, únicamente los valores que le lleguen desde el nuevo arranque. En otra consola, generar una secuencia de números del 1 al 30 (utilizando el comando "seq") y enviársela mediante una tubería o pipe (indicada con el carácter "|") a un productor sobre el topic "testOrder1". Observar como se consume dicha secuencia en el consumidor.

Comandos:
- `bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic testOrder1`
- `seq 1 30 | bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testOrder1` 

Lo que ha impreso en la consola del consumidor:
```
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
```

### 18. Arrancar un consumidor sobre el topic "testOrder2", sin que consuma desde el principio, únicamente los valores que le lleguen desde el nuevo arranque. En otra consola, generar una secuencia de números del 1 al 30 (utilizando el comando "seq") y enviársela mediante una tubería o pipe (indicada con el carácter "|") a un productor sobre el topic "testOrder2". Observar como se consume dicha secuencia en el consumidor.

> Nota: Para emular este resultado, he tenido que degradar la versión de kafka a la 2.3.1

Comando:
- `bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic testOrder2`
- `seq 1 30 | bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testOrder2` 

Lo que ha impreso en la consola del consumidor:
```
2
5
8
11
14
17
20
23
26
29
3
6
9
12
15
18
21
24
27
30
1
4
7
10
13
16
19
22
25
28

```

### 19. ¿Cuáles son las diferencias entre lo mostrado en el consumidor en los dos ejercicios anteriores? ¿A qué se deben?

La diferencia es que, en el caso del producer-consumer del topic testOrder1, los resultados llegan ordenados, del 1 al 30. En el caso del producer-consumer del topic testOrder2, llegan desordenados.

Esto se debe a que el orden se mantiene solo dentro de las particiones.

### 20. Obtener los últimos offsets de los mensajes del topic "testOrder1" y "testOrder2". ¿Qué se observa?. (Utilizar el comando kafka-run-class con la opción kafka.tools.GetOffsetShell)

Comandos:
- ` bin/kafka-run-class.sh kafka.tools.GetOffsetShell --broker-list localhost:9092,localhost:9093,localhost:9094 --topic testOrder1`

Resultado:
```
testOrder1:0:33
```

Comandos:
- ` bin/kafka-run-class.sh kafka.tools.GetOffsetShell --broker-list localhost:9092,localhost:9093,localhost:9094 --topic testOrder2`

Resultado:
```
testOrder2:0:102
testOrder2:1:40
testOrder2:2:71
```

Conclusión: Se oberva que el topic testOrder2 tiene 3 offset, pues el offset es el ordinal que corresponde a un evento dentro de una partición. Y éste tiene tres particiones. En cambio, en testOrder1, al solo haber una partición, solamente hay un offset.

### 21. Detener de forma no planificada (utilizando Ctrl-C o cerrando la consola) el broker con identificador 2, y describir el topic "testOrder2". ¿Cuáles son las diferencias con la descripción que obtuvimos en el ejercicio 14? ¿A qué se deben?

- Comando: `bin/kafka-topics.sh --describe --bootstrap-server localhost:9092,localhost:9093,localhost:9094 --topic testOrder2`
- Resultado:

```Topic:testOrder2	PartitionCount:3	ReplicationFactor:3	Configs:segment.bytes=1073741824
	Topic: testOrder2	Partition: 0	Leader: 0	Replicas: 0,2,1	Isr: 1,0
	Topic: testOrder2	Partition: 1	Leader: 1	Replicas: 2,1,0	Isr: 1,0
	Topic: testOrder2	Partition: 2	Leader: 1	Replicas: 1,0,2	Isr: 1,0```

- Comentarios: Se puede comprobar que debido a la alta tolerancia a fallos de kafka, los líderes han sido reasignados a los brokers 1 y 2, las réplicas siguen iguales. No obstante, lás únicas réplicas actualizadas con el líder (isr) son los brokers 1 y 0. Pues el dos se ha detenido.